# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [1]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [2]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,穀物,こくもつ
1,歓迎,かんげい
2,済ませる,すませる
3,老い,おい
4,婚約,こんやく


#### load Models

In [3]:
azure_llm = AzureChatOpenAI(
    azure_endpoint="https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
    api_key=os.environ["AZURE_API_KEY"],
    model_name="gpt-4o",
    api_version="2025-01-01-preview",
    temperature=0.5,
)

In [4]:
aws_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.3),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

## Question Explore

# Kanji 读假名（读音问题）

In [ ]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
問題 3  
つぎの文章を読んで、文章全体の内容を考えて、19 から 22 の中に入る最もよいものを、1・2・3・4 から一つえらびなさい。  
  
以下は、留学生の作文である。  
  
---  
  
夏休みの思い出  
  
お母さん、中学生の妹さんと住んでいます。日本人の家に泊まるのは初めてだったので、行く前は少し不安な気持ちもありました。[20] 行ってみたらとても楽しかったです。  
  
印象に残っているのは、巡りの畑で育てた野菜を使って、みんなで料理を作ったことです。友達のお母さんは、畑でいろいろな野菜を育てていました。私たちは、その野菜を使ってみんなで料理をしました。私は、お店に売られている野菜 [21] 食べたことがありませんでした。家で育てた野菜を食べたのは初めてでしたが、とてもおいしかったです。特に「私も野菜を育ててみたい」と、胸がいっぱ言う行ってらない。」と言ったら、それを聞いていたお母さんが、家の中でも育てることができる野菜について教えてくれました。  
  
お母さんに教えてもらったやり方で、私も野菜を [22]。今、２種類の野菜を育てています。  
  
野菜の世話をしながら、楽しかった夏休みのことをいつも思い出しています。  
  
--- 

19.  
1. 招待してくれたのです    
2. 招待してくれたはずです    
3. 招待してくれたばかりです    
4. 招待してくれたそうです    
  
20.  
1. それで    
2. でも    
3. 実は    
4. また    
  
21.  
1. は    
2. などを    
3. しか    
4. だけ    
  
22.  
1. 育ててみてほしいです    
2. 育ててみてもいいです    
3. 育ててみようとしました    
4. 育ててみることにしました     
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = azure_llm.invoke(f"You are a japanese teacher. Your job is to write a paper for reading the article, fill in the gaps, choose the grammar structure that best fits the context from the following 4 options, and also test the ability to read the article, similar to cloze test, with four options per question. Only refer to the format, not the content."
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            穀物    こくもつ
1            歓迎    かんげい
2          済ませる    すませる
3            老い      おい
4            婚約    こんやく
...         ...     ...
2134         命令    めいれい
2135         注目   ちゅうもく
2136          夜       よ
2137         環境   かんきょう
2138         無視      むし

[2139 rows x 2 columns]


In [6]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [7]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で価格を尋ねる | レストランで食べ物を注文する | 交通手段について話す| 天気の状況について話す | 趣味について話す | 店で商品の特徴や価格を尋ねる | スーパーマーケットで買い物をする | レストランで食べ物や飲み物を注文する | カフェでテイクアウトを頼む | コンビニで支払い方法について質問する |  ネットショッピングで商品を選ぶ"})
display(Markdown(kanji["question"]))

# Formal Exam Paper: Cloze Test  
**Topic:** 夏休みの思い出  

---

## 問題 3  
つぎの文章を読んで、文章全体の内容を考えて、19 から 22 の中に入る最もよいものを、1・2・3・4 から一つえらびなさい。  

以下は、留学生の作文である。  

---

### 夏休みの思い出  

お母さん、中学生の妹さんと住んでいます。日本人の家に泊まるのは初めてだったので、行く前は少し不安な気持ちもありました。[20] 行ってみたらとても楽しかったです。  

印象に残っているのは、巡りの畑で育てた野菜を使って、みんなで料理を作ったことです。友達のお母さんは、畑でいろいろな野菜を育てていました。私たちは、その野菜を使ってみんなで料理をしました。私は、お店に売られている野菜 [21] 食べたことがありませんでした。家で育てた野菜を食べたのは初めてでしたが、とてもおいしかったです。特に「私も野菜を育ててみたい」と、胸がいっぱ言う行ってらない。」と言ったら、それを聞いていたお母さんが、家の中でも育てることができる野菜について教えてくれました。  

お母さんに教えてもらったやり方で、私も野菜を [22]。今、２種類の野菜を育てています。  

野菜の世話をしながら、楽しかった夏休みのことをいつも思い出しています。  

---

### 選択肢  

#### 19.  
1. 招待してくれたのです    
2. 招待してくれたはずです    
3. 招待してくれたばかりです    
4. 招待してくれたそうです    

#### 20.  
1. それで    
2. でも    
3. 実は    
4. また    

#### 21.  
1. は    
2. などを    
3. しか    
4. だけ    

#### 22.  
1. 育ててみてほしいです    
2. 育ててみてもいいです    
3. 育ててみようとしました    
4. 育ててみることにしました    

---

## 正解と解説  

### 19. **正解: 1. 招待してくれたのです**  
**解説:**  
この選択肢は、文章全体の流れに最も合っています。「招待してくれたのです」は、事実を述べる表現であり、読者に状況を自然に伝える役割を果たします。他の選択肢は推測や時制が合わないため不適切です。  

**主な挑戦:**  
候補者需要理解「のです」的用法，表达事实或解释的语气。其他选项如“招待してくれたはずです”更偏向推测，可能会误导候选者。  

---

### 20. **正解: 2. でも**  
**解説:**  
「でも」は、前後の文の対比を示す接続詞です。前文では「少し不安な気持ちもありました」と述べているため、「でも」が最も適切です。他の選択肢は文章の流れに合わず、不自然です。  

**主な挑戦:**  
候选者需要判断接续词的逻辑关系，特别是对比关系。选项“それで”更适合因果关系，“実は”更适合强调内容，不符合上下文。  

---

### 21. **正解: 3. しか**  
**解説:**  
「しか」は否定形とともに使われ、「～しか～ない」という形で限定を表します。文章中「お店に売られている野菜しか食べたことがありませんでした」は自然且符合语法的表达方式。  

**主な挑戦:**  
候选者需要理解「しか」的否定限定用法，并正确搭配否定句。其他选项如“は”更偏向一般陈述，“などを”表示列举，不符合语境。  

---

### 22. **正解: 4. 育ててみることにしました**  
**解説:**  
「育ててみることにしました」は、意志を表す表現であり、文章の流れとして、「お母さんに教えてもらったやり方で」という背景を受けて、行動を決定したことを自然に示しています。他の選択肢は意志や決定のニュアンスが弱いため不適切です。  

**主な挑戦:**  
候选者需要区分意志表达与建议或可能性的差异。选项“育ててみてもいいです”更偏向许可，不符合情境。  

---

### Markdown Beautification  

```markdown
# Formal Exam Paper: Cloze Test  
**Topic:** 夏休みの思い出  

---

## 問題 3  
つぎの文章を読んで、文章全体の内容を考えて、19 から 22 の中に入る最もよいものを、1・2・3・4 から一つえらびなさい。  

以下は、留学生の作文である。  

---

### 夏休みの思い出  

お母さん、中学生の妹さんと住んでいます。日本人の家に泊まるのは初めてだったので、行く前は少し不安な気持ちもありました。[20] 行ってみたらとても楽しかったです。  

印象に残っているのは、巡りの畑で育てた野菜を使って、みんなで料理を作ったことです。友達のお母さんは、畑でいろいろな野菜を育てていました。私たちは、その野菜を使ってみんなで料理をしました。私は、お店に売られている野菜 [21] 食べたことがありませんでした。家で育てた野菜を食べたのは初めてでしたが、とてもおいしかったです。特に「私も野菜を育ててみたい」と、胸がいっぱ言う行ってらない。」と言ったら、それを聞いていたお母さんが、家の中でも育てることができる野菜について教えてくれました。  

お母さんに教えてもらったやり方で、私も野菜を [22]。今、２種類の野菜を育てています。  

野菜の世話をしながら、楽しかった夏休みのことをいつも思い出しています。  

---

### 選択肢  

#### 19.  
1. 招待してくれたのです    
2. 招待してくれたはずです    
3. 招待してくれたばかりです    
4. 招待してくれたそうです    

#### 20.  
1. それで    
2. でも    
3. 実は    
4. また    

#### 21.  
1. は    
2. などを    
3. しか    
4. だけ    

#### 22.  
1. 育ててみてほしいです    
2. 育ててみてもいいです    
3. 育ててみようとしました    
4. 育ててみることにしました    

---

## 正解と解説  

### 19. **正解: 1. 招待してくれたのです**  
**解説:**  
この選択肢は、文章全体の流れに最も合っています。「招待してくれたのです」は、事実を述べる表現であり、読者に状況を自然に伝える役割を果たします。他の選択肢は推測や時制が合わないため不適切です。  

**主な挑戦:**  
候補者需要理解「のです」的用法，表达事实或解释的语气。其他选项如“招待してくれたはずです”更偏向推测，可能会误导候选者。  

---

### 20. **正解: 2. でも**  
**解説:**  
「でも」は、前後の文の対比を示す接続詞です。前文では「少し不安な気持ちもありました」と述べているため、「でも」が最も適切です。他の選択肢は文章の流れに合わず、不自然です。  

**主な挑戦:**  
候选者需要判断接续词的逻辑关系，特别是对比关系。选项“それで”更适合因果关系，“実は”更适合强调内容，不符合上下文。  

---

### 21. **正解: 3. しか**  
**解説:**  
「しか」は否定形とともに使われ、「～しか～ない」という形で限定を表します。文章中「お店に売られている野菜しか食べたことがありませんでした」は自然且符合语法的表达方式。  

**主な挑戦:**  
候选者需要理解「しか」的否定限定用法，并正确搭配否定句。其他选项如“は”更偏向一般陈述，“などを”表示列举，不符合语境。  

---

### 22. **正解: 4. 育ててみることにしました**  
**解説:**  
「育ててみることにしました」は、意志を表す表現であり、文章の流れとして、「お母さんに教えてもらったやり方で」という背景を受けて、行動を決定したことを自然に示しています。他の選択肢は意志や決定のニュアンスが弱いため不適切です。  

**主な挑戦:**  
候选者需要区分意志表达与建议或可能性的差异。选项“育ててみてもいいです”更偏向许可，不符合情境。  
```